In [1]:
!pip install torch scikit-learn --quiet

In [2]:
# !pip install torch scikit-learn --quiet

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# --- Data ---
reviews = [
    "I love this product, it works perfectly!",
    "Worst quality ever, very disappointed.",
    "Absolutely amazing, exceeded my expectations!",
    "This is terrible, don’t buy it.",
    "Great quality and fast delivery!",
    "Not worth the money at all."
]
labels = [1, 0, 1, 0, 1, 0]

# --- Vectorize Text ---
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(reviews).toarray()
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(labels, dtype=torch.float32)

# --- Model ---
class SentimentNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.fc(x)

model = SentimentNet(X.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# --- Train ---
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X_tensor).squeeze()
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()

print(f"Final training loss: {loss.item():.4f}")

# --- Predict ---
test = ["Amazing quality!", "Not good at all."]
test_X = vectorizer.transform(test).toarray()
test_tensor = torch.tensor(test_X, dtype=torch.float32)
preds = model(test_tensor).detach().numpy()

for t, p in zip(test, preds):
    print(f"{t} ➝ {'Positive' if p>0.5 else 'Negative'} ({p[0]:.2f})")


Final training loss: 0.0024
Amazing quality! ➝ Positive (0.83)
Not good at all. ➝ Negative (0.03)


# Bag-of-Words encodes text into numeric form.

# Neural net predicts sentiment.

# PyTorch gives full control — ideal for research.